In [ ]:
"""
TODO:
- Get at least a reasonable looking change. Maybe start with bad initial conditions.
"""
import project_path
from model.neural_model import NeuralModel
import connectomes
import distance
import dynamics

import matplotlib.pyplot as plt
import numpy as np
import pdb

N = 3
simul_ts = 1000
eval_ts = 500
dt = 0.01
simul_timepoints = np.arange(0, simul_ts * dt, dt)
eval_timepoints = np.arange((simul_ts - eval_ts)*dt, simul_ts * dt, dt)

# Golden dynamics

In [ ]:
# Initial run
gold_dyn = dynamics.get_jimin_3neuron_dynamics(simul_ts, dt)
top_mode_gold_dyn = dynamics.get_top_mode(gold_dyn)
preprocessed_gold_dyn = distance.preprocess_pop_dyn(gold_dyn, eval_ts)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(simul_timepoints, top_mode_gold_dyn)
fig.suptitle("Top mode of gold dyn (raw)")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(eval_timepoints, preprocessed_gold_dyn)
fig.suptitle("Preprocessed gold dynamics to evaluate against")
_ = _

# Initial unoptimized dynamics

In [ ]:
init_Gg, init_Gs, is_inhibitory = connectomes.get_random_connectome(N)
I_ext = dynamics.get_jimin_3neuron_Iext()
initial_dyn = dynamics.run_neural_model(N, init_Gg, init_Gs, is_inhibitory, I_ext, simul_ts, dt)

top_mode_initial_dyn = dynamics.get_top_mode(initial_dyn)
preprocessed_initial_dyn = distance.preprocess_pop_dyn(initial_dyn, eval_ts)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(simul_timepoints, top_mode_initial_dyn)
fig.suptitle("Top mode of initial dyn (raw)")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(eval_timepoints, preprocessed_gold_dyn, label="gold", c="gold")
ax.plot(eval_timepoints, preprocessed_initial_dyn, label="actual")
ax.legend()
fig.suptitle("Comparison of preprocessed dyns against gold")

error = distance.ts_distance_euclidean(preprocessed_gold_dyn, preprocessed_initial_dyn)
print("Initial Gg = %s\n Gs = %s\n" % (init_Gg, init_Gs))
print("Initial error = " + str(error))
_ = _

# Perform optimization

In [ ]:
def create_obj_fun(N, preprocessed_gold_dyn, eval_ts):
  num_called = 0
  min_error = 1000000
  def obj_fun(compact_vec):
    nonlocal num_called
    nonlocal min_error
    global init_Gg
    global init_Gs
    num_called += 1
    gg_mat, gs_mat = connectomes.compact_to_model_param(compact_vec, N)
    pop_dyn = dynamics.run_neural_model(N, gg_mat, gs_mat, is_inhibitory, I_ext, simul_ts, dt)
    preprocessed_pop_dyn = distance.preprocess_pop_dyn(pop_dyn, eval_ts)
    error = distance.ts_distance_euclidean(preprocessed_gold_dyn, preprocessed_pop_dyn)
    min_error = min(min_error, error)
    if num_called % 10 == 0:
      print("Evaluation %s, error = %.2f, min_error = %.2f" % (num_called, error, min_error))
      print("Gg = " + str(gg_mat))
      print("Gs = " + str(gs_mat))
      print("delta Gg = " + str(init_Gg - gg_mat))
      print("delta Gs = " + str(init_Gs - gs_mat))
    return error
  return obj_fun

In [ ]:
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import time

init_cond_compact = connectomes.model_to_compact_param(init_Gg, init_Gs, N)
obj_fun = create_obj_fun(N, preprocessed_gold_dyn, eval_ts)
bnds = [(0, 10)] * len(init_cond_compact)

# See the options from here
# https://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.optimize.show_options.html
def optimize_with_SLSQP():
  return minimize(obj_fun, init_cond_compact, method='SLSQP', bounds=bnds,
                  options={'maxiter':2})

def optimize_with_L_BFGS_B():
  return minimize(obj_fun, init_cond_compact, method='L-BFGS-B', bounds=bnds,
                  options={'maxiter':2})

def optimize_with_Powell():
  options = {}
  # options={'maxfev':2}
  return minimize(obj_fun, init_cond_compact, method='Powell', bounds=bnds, options=options)

def optimize_with_Nelder_mead():
  options = {}
  # options={'maxfev':2}
  return minimize(obj_fun, init_cond_compact, method='Nelder-Mead', bounds=bnds, options=options)


def optimize_with_basin_hopping():
  minimizer_kwargs = {"method":"Powell", "bounds":bnds}
  # You can add niter=k to limit the number of bruteforces
  return basinhopping(obj_fun, init_cond_compact, minimizer_kwargs=minimizer_kwargs)
  
start_time = time.time()
res = optimize_with_basin_hopping()
print("Total optimization time = %.2fs" % (time.time() - start_time))
print(res)

In [ ]:
new_Gg, new_Gs = connectomes.compact_to_model_param(res.x, N)
print("The optimized Gg and Gs are:\n%s\n%s\n" % (new_Gg, new_Gs))
print("Old Gg and Gs are:\n%s\n%s\n" % (init_Gg, init_Gs))
print("The difference matrices are:\n%s\n%s\n" % (new_Gg-init_Gg, new_Gs-init_Gs))


# Compare optimized dynamics against golden

In [ ]:
# Plot the optimized results

opt_dyn = dynamics.run_neural_model(N, new_Gg, new_Gs, is_inhibitory, I_ext, simul_ts, dt)

top_mode_opt_dyn = dynamics.get_top_mode(opt_dyn)
preprocessed_opt_dyn = distance.preprocess_pop_dyn(opt_dyn, eval_ts)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(simul_timepoints, top_mode_opt_dyn)
fig.suptitle("Top mode of opt dyn (raw)")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
ax.plot(eval_timepoints, preprocessed_gold_dyn, label="gold", c="gold")
ax.plot(eval_timepoints, preprocessed_opt_dyn, label="actual")
ax.legend()
fig.suptitle("Comparison of preprocessed dyns against gold, optimized")

error = distance.ts_distance_euclidean(preprocessed_gold_dyn, preprocessed_opt_dyn)
print("Optimized error = " + str(error))
_ = _